# CX 4230, Spring 2016: [22] Input modeling

This notebook includes sample code to accompany the slides from the Monday, February 29 class. It does not contain any exercises.

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X = np.array ([105.84, 28.92, 98.64, 55.64,
               128.04, 45.60, 67.80, 105.12,
               48.48, 51.84, 173.40, 51.96,
               54.12, 68.64, 93.12, 68.88,
               84.12, 68.64, 41.52, 127.92,
               42.12, 17.88, 33.00])
print (len (X), "observations:")
print (X)

For the next code cell, refer to the documentation for Scipy's [`linregress()`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html).

In [ ]:
from scipy.stats import linregress

T = np.arange (len (X))
slope, intercept, rvalue, pvalue, stderr = linregress (T, X)

print ("Slope:", slope)
print ("Intercept:", intercept)
print ("p-value:", pvalue)

For the next code cell, refer to the documentation for Numpy's [`pad()`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html#numpy.pad) function.

In [ ]:
# Running means (fixed w)
w = 2 # window size
n, r = len (X), len (X) % w

n_w = (n-r) / w
X_w = np.reshape (X if r == 0 else X[:-r], (n_w, w))
np.pad (X_w, ((0, 0), (0, 1)), 'mean')

In [ ]:
def calc_windowed_mean (X, w):
    n, r = len (X), len (X) % w
    n_w = (n - r) / w
    if r == 0:
        X_w = np.reshape (X, (n_w, w))
    else:
        X_w = np.reshape (X[:-r], (n_w, w))
        
    # Add column of mean values
    X_w = np.pad (X_w, ((0, 0), (0, 1)), 'mean')
    T_w = np.arange (0, n-r, w) + w/2
    return X_w, T_w

# Demo
calc_windowed_mean (X, 2)

In [ ]:
fig = plt.figure (figsize=(18, 6))
ax = fig.add_subplot (111)
for w in range (1, len (X)+1, 5):
    X_w, T_w = calc_windowed_mean (X, w)
    xp, yp = T_w, X_w[:, -1:]
    ax.plot (xp, yp, 'o:', label=str (w))
ax.legend ()

In [ ]:
def sample_mean (X):
    return np.mean (X)

sample_mean (X)

In [ ]:
def sample_autocovar (X, h):
    n = len (X)
    n_h = n - abs (h)
    X_t = X[:n_h]
    X_t_h = X[abs (h):n]
    mu = sample_mean (X)
    return np.sum ((X_t_h - mu) * (X_t - mu)) / n

# Demo
sample_autocovar (X, 3)

In [ ]:
def sample_autocorr (X, h=None):
    n = len (X)
    if h is not None:
        assert abs (h) < n
        return sample_autocovar (X, h) / sample_autocovar (X, 0)
    else:
        C = np.zeros (2*n-1)
        H = np.arange (-(n-1), n)
        for h in H:
            C[n-1+h] = sample_autocorr (X, h)
        return C, H
    assert False

# Demo
sample_autocorr (X)

In [ ]:
def viz_autocorr (X):
    C, H = sample_autocorr (X)
    
    fig = plt.figure (figsize=(18, 6))
    ax = fig.add_subplot (111)
    ax.stem (H, C, '-.')
    plt.title ('Lag autocorrelations')
    ax.set_xlabel ('Lag')
    
    return fig, ax, C, H

# Demo
_, _, _, _ = viz_autocorr (X)

The following code cell shows an alternative way to implement the sample autocorrelation measure using Scipy's built-in [`correlate()`](http://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate.html) function.

In [ ]:
from scipy.signal import correlate

def sample_autocorr2 (X, h=None):
    n = len (X)    
    mu_X = np.mean (X)
    Y = correlate ((X - mu_X)/n, (X - mu_X)/n)
    C = Y / Y[int (len (Y)/2)]
    H = np.arange (-(n-1), n)
    
    if h is not None:
        assert -n < h < n
        return C[-(n-1)+h]
    else:
        return C, H

def viz_autocorr2 (X):
    C, H = sample_autocorr2 (X)
    fig = plt.figure (figsize=(18, 6))
    ax = fig.add_subplot (111)
    ax.stem (H, C, '-.')
    plt.title ('Lag autocorrelations (Method 2)')
    ax.set_xlabel ('Lag')
    return fig, ax, C, H

# Demo
_, _, _, _ = viz_autocorr2 (X)